# Predicting the past, United States

## Introduction

In the United States, 112 commercial reactors have been deployed and operated since 1967. Predicting the past repository uses published data of those commercial reactors and simulates these reactors using _CYCLUS_, an agent-based simulation software, to compare simulation results with the real world statistics such as power generated. These simulations can be used as baseline comparison models for studies that involve simulations modeling the future which include the DOE-NE's study to screen fuel cycle options. Information regarding reactors were obtained from Power Reactor Information System's (PRIS) reactor database. [reactors_pris_2016.csv](import_data/reactors_pris_2016.csv)

## Obtaining Recipes for Simulation

First, a input file for _CYCLUS_ that specifies information such as the simulation start year, reactor deployment,  facility deployment, and recipes for fresh and spent nuclear fuel (SNF) was produced. The recipes for fresh and Spent Nuclear Fuel (SNF) were obtained from `vision_recipes.xls` spreadsheet file, which contained composition data for fresh and spent nuclear fuel for uranium oxide (UOX) and mixed oxide (MOX) fuels at different burn ups.

Importing raw composition data from `vision_recipes.xls` was performed through a python script and was demonstrated below. Due to limited information about the reactors, all reactors were assumed to operate at a burn up of 51 GWd/MTHM.

### Demonstration
__Import necessary libraries__

In [ ]:
import csv
import collections
import dateutil.parser as date
import import_fleetcomp as idata
import jinja2
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import sys
import sqlite3 as lite
from matplotlib import cm
from pyne import nucname as nn
from IPython.display import Image

sys.path.append('analysis/')
import analysis as an

region = 'UNITED_STATES'

__Read fuel recipe data__

Contents from `vision_recipes.xls` was read and stored in a list object by looping over row in the spreadsheet file.

In [ ]:
def import_csv(in_csv, delimit):
    """ Imports contents of a comma delimited csv file
    to a 2D list.

    Parameters
    ---------
    in_csv: str
        csv file name.
    delimit: str
        delimiter of the csv file

    Returns
    -------
    data_list: list
        list with fleetcomp data.
    """
    with open(in_csv, encoding='utf-8') as source:
        sourcereader = csv.reader(source, delimiter=delimit)
        data_list = []
        for row in sourcereader:
            data_list.append(row)
    return data_list


recipes = import_csv('import_data/vision_recipes/uox.csv', ',')
print(*recipes[2:6], '.', '.', '.', sep='\n')

__Load template for _CYCLUS_ input file__

A jinja template file, which was used to produce the _CYCLUS_ simulation input file was loaded into memory.

In [ ]:
def load_template(in_template):
    """ Returns a jinja2 template.

    Parameters
    ---------
    in_template: str
        template file name.

    Returns
    -------
    output_template: jinja template object
    """
    with open(in_template, 'r') as default:
        output_template = jinja2.Template(default.read())
    return output_template


recipe_template = load_template('templates/recipes_template.xml')

__Select appropriate recipe for a given burnup__

The appropriate composition for fresh and spent nuclear fuel was selected with the specified burnup. The composition data was then stored into a dictionary object with isotope as key, and mass fraction composition as value.

In [ ]:
def get_composition_fresh(in_list, burnup):
    """ Returns a dictionary of isotope and composition (in mass fraction)
    using vision_recipes for fresh UOX

    Parameters
    ---------
    in_list: list
        list file containing vision_recipes data.
    burnup: int
        burnup

    Returns
    -------
    data_dict: dictionary
        dictionary with key: isotope, and value: composition.
    """
    data_dict = {}
    for i in range(len(in_list)):
        if i > 1:
            if burnup == 33:
                data_dict.update({nn.id(in_list[i][0]):
                                  float(in_list[i][1])})
            elif burnup == 51:
                data_dict.update({nn.id(in_list[i][0]):
                                  float(in_list[i][3])})
            else:
                data_dict.update({nn.id(in_list[i][0]):
                                  float(in_list[i][5])})
    return data_dict


def get_composition_spent(in_list, burnup):
    """ Returns a dictionary of isotope and composition (in mass fraction)
    using vision_recipes for spent nuclear fuel

    Parameters
    ---------
    in_list: list
        list file containing vision_recipes data.
    burnup: int
        burnup

    Returns
    -------
    data_dict: dictionary
        dictionary with key: isotope, and value: composition.
    """
    data_dict = {}
    for i in range(len(in_list)):
        if i > 1:
            if burnup == 33:
                data_dict.update({nn.id(in_list[i][0]):
                                  float(in_list[i][2])})
            elif burnup == 51:
                data_dict.update({nn.id(in_list[i][0]):
                                  float(in_list[i][4])})
            else:
                data_dict.update({nn.id(in_list[i][0]):
                                  float(in_list[i][6])})
    return data_dict


fresh = get_composition_fresh(recipes, 51)
spent = get_composition_spent(recipes, 51)
print('Spent Fuel Composition:')
print('ISOTOPE:   Massfrac')
for k in list(spent.keys())[:3]:
    print(k, ': ', spent[k])
print('.', '.', '.', sep='\n')

__Render recipe template with recipe data__

`jinja2` library was used to render recipe data to the previously loaded `recipe_template`. `pyne` converted the isotope name to a _CYCLUS_ readable format. Code then saved the rendered information to an xml file for use with _CYCLUS_.

In [ ]:
def write_recipes(fresh_dict, spent_dict, in_template, burnup, region):
    """ Renders jinja template using fresh and spent fuel composition and
    outputs an xml file containing recipe data

    Parameters
    ---------
    fresh_dict: dictionary
        dictionary with key=isotope, and value=composition for fresh UOX
    spent_dict: dictionary
        dictionary with key=isotope, and value=composition for spent fuel
    in_template: jinja template object
        jinja template object to be rendered.
    burnup: int
        amount of burnup

    Returns
    -------
    null
        generates reactor files for cyclus.
    """
    out_path = 'cyclus/input/' + region + '/recipes/'
    pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)
    rendered = in_template.render(fresh=fresh_dict,
                                  spent=spent_dict)
    with open(out_path + '/uox_' +
              str(burnup) + '.xml', 'w') as output:
        output.write(rendered)


write_recipes(fresh, spent, recipe_template, 33, region)
write_recipes(fresh, spent, recipe_template, 51, region)
write_recipes(fresh, spent, recipe_template, 100, region)

[uox_51.xml](cyclus/input/US/recipes/uox_51.xml) is a sample rendered result that contains fresh and spent nuclear fuel compositions. 

__recipe template__

`jinja2` library was used to produce  _CYCLUS_ input files from templates. The library replaced simulation-specific information, such as the recipe for SNF, to the template file. This allowed end-users to produce _CYCLUS_ simulation inputs that vary in length depending on the data with which it has been rendered. An example is shown below.

Recipe template file:

```
    <recipes>
      <recipe>
        <name>fresh_uox</name>
        <basis>atom</basis>
        <nuclide>
          <id>922350000</id>
          <comp>4.5</comp>
        </nuclide>
        <nuclide>
          <id>922380000</id>
          <comp>95.5</comp>
        </nuclide>
      </recipe>
      <recipe>
        <name>spent_uox</name>
        <basis>atom</basis>
        {% for key, value in vision.items() -%}
        <nuclide>  <id>{{ key }}</id>  <comp>{{ value }}</comp>  </nuclide>
        {% endfor -%}
      </recipe>
    </recipes>
```

The template shown above was a recipe template for fresh and spent nuclear fuel. `jinja2` recognized the for-loop in the following lines 
```
    {% for key, value in vision.items() -%}`
    <nuclide>  <id>{{ key }}</id>  <comp>{{ value }}</comp>  </nuclide>
    {% endfor -%}
```
and iterated over the composition dictionary to render the isotope and its composition.

[uox_51.xml](cyclus/input/US/recipes/uox_51.xml) file shows the rendered result. 

## Obtaining Deployment Data

Reactors specified in [reactors_pris_2016.csv](import_data/reactors_pris_2016.csv), needed to be properly imported. The reactors should be deployed at the correct timesteps for an accurate simulation. The spreadsheet file contained reactor information such as name, deployment date, net capacity, and deployment nation. Obtaining information regarding reactors for _CYCLUS_ was also performed using a set of python functions. The same generic steps were repeated for this demonstration: importing data stored in a delimited text file, and rendering the imported data to a _CYCLUS_ template file.

### Demonstration
__Read fleetcomp reactor data__

The `import_csv` function was also used for this application any other examples below that required data stored in a spreadsheet file to be imported for processing and rendering.

In [ ]:
pris = import_csv('import_data/reactors_pris_2016.csv', ',')

print(*pris[1:6], sep='\n')
print('.', '.', '.', sep='\n')

__Select reactors in the specified region__

The PRIS reactor data spreadsheet file contained reactors from all nations. Thus, in order to obtain the reactors deployed in the United States, a separate function was written that selects reactors from the US by searching for a matching nation.

In [ ]:
def select_region(in_list, region):
    """ Returns a list of reactors that have a start_date
    and are note experimental

    Parameters
    ----------
    in_list: list
            imported csv file in list format
    region: str
            name of the region

    Returns
    -------
    reactor_list: list
            list of reactors from PRIS
    """
    UNITED_STATES = {'UNITED STATES'}
    SOUTH_AMERICA = {'ARGENTINA', 'BRAZIL'}
    NORTH_AMERICA = {'CANADA', 'MEXICO', 'UNITED STATES'}
    EUROPE = {'BELARUS', 'BELGIUM', 'BULGARIA',
              'CZECHOSLOVAKIA', 'FINLAND', 'FRANCE',
              'GERMANY', 'ITALY', 'NETHERLANDS',
              'POLAND', 'ROMANIA', 'RUSSIA',
              'SLOVENIA', 'SOVIET UNION', 'SPAIN',
              'SWEDEN', 'SWITZERLAND', 'TURKEY',
              'UKRAINE', 'UNITED KINGDOM'
              }
    ASIA = {'BANGLADESH', 'CHINA', 'INDIA',
            'IRAN', 'JAPAN', 'KAZAKHSTAN',
            'PAKISTAN', 'PHILIPPINES', 'SOUTH KOREA',
            'UNITED ARAB EMIRATES', 'VIETNAM'}
    AFRICA = {'EGYPT', 'MOROCCO', 'SOUTH AFRICA', 'TUNISIA'}
    ALL = SOUTH_AMERICA | NORTH_AMERICA | EUROPE | ASIA | AFRICA | UNITED_STATES
    regions = {'SOUTH_AMERICA': SOUTH_AMERICA, 'NORTH_AMERICA': NORTH_AMERICA,
               'ASIA': ASIA, 'AFRICA': AFRICA, 'EUROPE': EUROPE, 
               'UNITED_STATES': UNITED_STATES, 'ALL': ALL}

    if region.upper() not in regions.keys():
        raise ValueError(region + 'is not a valid region')
    reactor_list = []
    for row in in_list:
        country = row[0]
        if country.upper() in regions[region.upper()]:
            start_date = row[9]
            if start_date.strip():
                reactor_list.append(row)
    return reactor_list

reactor_list = select_region(pris, region)
print(*reactor_list[:4], '.', '.', '.', sep='\n')

__Select and render useful reactor information__

In this step, the following information was obtained from the PRIS spreadsheet: reactor name, type, deployed nation, and capacity. Other information, such as number of assemblies, assemblies per batch, mass of core, and mass of assembly was obtained from published sources [1], [2]. Then, `write_reactors` function was used to save these information as _CYCLUS_ reactor specifications. The function looped over each row (reactor) and checked the type of reactor and its capacity. It was assumed that reactors with capacity less than 400 MWe were experimental reactors. Then, the function obtained all the raw and derived variables necessary for simulation and rendered the information to the template.

In [ ]:
def write_reactors(in_list, out_path, reactor_template):
    """ Obtains information regarding reactors
    and renders the information into a jinja template

    Parameters
    ----------
    in_list: list
        list containing PRIS data
    out_path: str
        output path for reactor files
    reactor_template: str
        path to reactor template

    Returns
    -------
    null
        writes xml files containing information about a reactor
    """
    if out_path[-1] != '/':
        out_path += '/'
    pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)
    reactor_template = idata.load_template(reactor_template)
    for row in in_list:
        capacity = float(row[3])
        if capacity >= 400:
            name = row[1].replace(' ', '_')
            assem_per_batch = 0
            assem_no = 0
            assem_size = 0
            reactor_type = row[2]
            if reactor_type in ['BWR', 'ESBWR']:
                assem_no = 732
                assem_per_batch = assem_no / 3
                assem_size = 138000 / assem_no
            elif reactor_type in ['GCR', 'HWGCR']:  # Need batch number
                assem_no = 324
                assem_per_batch = assem_no / 3
                assem_size = 114000 / assem_no
            elif reactor_type == 'HTGR':  # Need batch number
                assem_no = 3944
                assem_per_batch = assem_no / 3
                assem_size = 39000 / assem_no
            elif reactor_type == 'PHWR':
                assem_no = 390
                assem_per_batch = assem_no / 45
                assem_size = 80000 / assem_no
            elif reactor_type == 'VVER':  # Need batch number
                assem_no = 312
                assem_per_batch = assem_no / 3
                assem_size = 41500 / assem_no
            elif reactor_type == 'VVER-1200':  # Need batch number
                assem_no = 163
                assem_per_batch = assem_no / 3
                assem_size = 80000 / assem_no
            else:
                assem_no = 241
                assem_per_batch = assem_no / 3
                assem_size = 103000 / assem_no

            rendered = reactor_template.render(name=name,
                                               lifetime=get_lifetime(row),
                                               assem_size=assem_size,
                                               n_assem_core=assem_no,
                                               n_assem_batch=int(
                                                   assem_per_batch),
                                               power_cap=row[3])
            with open(out_path + name.replace(' ', '_') + '.xml',
                      'w') as output:
                output.write(rendered)


def get_lifetime(in_list):
    """ Calculates the lifetime of a reactor using first
    grid data and shutdown date. Defaults to 720 if these
    data are not available

    Parameters
    ----------
    in_list: list
        single row from PRIS data that contains reactor
        information

    Returns
    -------
    lifetime: int
        lifetime of reactor
    """
    grid_date = in_list[9]
    shutdown_date = in_list[11]
    if not shutdown_date.strip():
        return 720
    else:
        n_days_month = 365.0 / 12
        delta = (date.parse(shutdown_date) - date.parse(grid_date)).days
        return int(delta / n_days_month)


out_path = 'cyclus/input/' + region + '/reactors'
reactor_template = 'templates/reactors_template.xml'
write_reactors(reactor_list, out_path, reactor_template)

[reactors](cyclus/input/UNITED_STATES/reactors) folder contained _CYCLUS_ input files for all reactors that were deployed in the simulation.

## Writing Deployment

In order to ensure proper deployment of all reactors during simulation, a separate function was written to calculate then save deployment information. The `build_time`, which was the number of timesteps taken to deploy the reactors, was calculated using the difference between the start date of the simulation and the reactor start date. Since _CYCLUS_ default unit of timestep is a month, the difference was also calculated in months.

### Demonstration 
__Read fleetcomp reactor data__

The `deploy_reactors` function was used for this application. The function imported the PRIS reactor file, produced the folder structure necessary for file output, and called `get_buildtime` function and `write_deployment` function. `get_buildtime` function parsed the deployment dates stored in the PRIS reactor file, and calculated the difference between the simulation start date and the reactor deployment date in months. Then, the resulting deployment month was stored in a dictionary object with the reactor name as key. and the deployed country and deployment month as value. Finally, `write_deployment` function uses the dictionary obtained from `get_buildtime` function to write _CYCLUS_ input file that specifies reactor deployment nation and timestep.

In [ ]:
def deploy_reactors(in_csv, region, start_year, deployinst_template,
                    inclusions_template, reactors_path, deployment_path):
    """ Generates xml files that specifies the reactors that will be included
    in a cyclus simulation.

    Parameters
    ---------
    in_csv: str
        csv file name.
    region: str
        region name
    start_year: int
        starting year of simulation
    deployinst_template: str
        path to deployinst template
    inclusions_template: str
        path to inclusions template
    reactors_path: str
        path containing reactor files
    deployment_path: str
        output path for deployinst xml

    Returns
    -------
    buildtime_dict: dict
        dictionary with key=[name of reactor], and
        value=[set of country and buildtime]
    """
    lists = []
    if reactors_path[-1] != '/':
        reactors_path += '/'
    for files in os.listdir(reactors_path):
        lists.append(reactors_path + files)
    in_data = idata.import_csv(in_csv, ',')
    reactor_list = select_region(in_data, region)
    buildtime = get_buildtime(reactor_list, start_year, lists)
    write_deployment(buildtime, deployment_path, deployinst_template,
                     inclusions_template)
    return buildtime


def get_buildtime(in_list, start_year, path_list):
    """ Obtains information regarding reactors that need to
    be deployed and renders the information into a jinja
    template

    Parameters
    ----------
    in_list: list
        list of reactors
    start_year: int
        starting year of simulation
    path_list: list
        list of paths to reactor files

    Returns
    -------
    buildtime_dict: dict
        dictionary with key=[name of reactor], and
        value=[set of country and buildtime]
    """
    buildtime_dict = {}
    for row in in_list:
        grid_date = date.parse(row[9])
        start_date = [grid_date.year, grid_date.month, grid_date.day]
        delta = ((start_date[0] - int(start_year)) * 12 +
                 (start_date[1]) +
                 round(start_date[2] / (365.0 / 12)))
        for index, reactor in enumerate(path_list):
            name = row[1].replace(' ', '_')
            country = row[0]
            file_name = (reactor.replace(
                os.path.dirname(path_list[index]), '')).replace('/', '')
            if (name + '.xml' == file_name):
                buildtime_dict.update({name: (country, delta)})
    return buildtime_dict


def write_deployment(in_dict, out_path, deployinst_template,
                     inclusions_template):
    """ Renders jinja template using dictionary of reactor name and buildtime
    and outputs an xml file that uses xinclude to include the reactors located
    in cyclus_input/reactors.

    Parameters
    ---------
    in_dict: dictionary
        dictionary with key: reactor name, and value: buildtime.
    out_path: str
        output path for files
    deployinst_template: str
        path to deployinst template
    inclusions_template: str
        path to inclusions template

    Returns
    -------
    null
        generates input files that have deployment and xml inclusions
    """
    if out_path[-1] != '/':
        out_path += '/'
    pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)
    deployinst_template = idata.load_template(deployinst_template)
    inclusions_template = idata.load_template(inclusions_template)
    country_list = {value[0] for value in in_dict.values()}
    for nation in country_list:
        temp_dict = {}
        for reactor in in_dict.keys():
            if in_dict[reactor][0].upper() == nation.upper():
                temp_dict.update({reactor: in_dict[reactor][1]})
        pathlib.Path(out_path + nation.replace(' ', '_') +
                     '/').mkdir(parents=True, exist_ok=True)
        deployinst = deployinst_template.render(reactors=temp_dict)
        with open(out_path + nation.replace(' ', '_') +
                  '/deployinst.xml', 'w') as output1:
            output1.write(deployinst)
    inclusions = inclusions_template.render(reactors=in_dict)
    with open(out_path + 'inclusions.xml', 'w') as output2:
        output2.write(inclusions)


pris_file = 'import_data/reactors_pris_2016.csv'
deployinst_tmpl = 'templates/' + region + '/deployinst_template.xml'
inclusions_tmpl = 'templates/inclusions_template.xml'
reactor_path = 'cyclus/input/' + region + '/reactors'
dployment_path = 'cyclus/input/' + region + '/buildtimes'
buildtime = deploy_reactors(pris_file, region, 1965, deployinst_tmpl,
                            inclusions_tmpl, reactor_path, dployment_path)

for k in list(buildtime.keys())[:4]:
    print(k, ': ', buildtime[k])
print('.', '.', '.', sep='\n')

[deployinst folder](cyclus/input/UNITED_STATES/buildtimes/) and [inclusions.xml](cyclus/input/UNITED_STATES/buildtimes/inclusions.xml) show the rendered result. 

`deployinst.xml` contained configurations for the _CYCAMORE::DeployInst_ archetype for the reactors in the simulation and `inclusions.xml` contained xml inclusions so that _CYCLUS_ can find and include the reactor files produced during __Obtaining Deployment Data__ step.

## XML inclusion and  Setting XML base for final input file

It was important to make sure that an xml base had been specified when xml inclusions through `XInclude` were used. `XInclude` was an easy way to include xml files within other xml files. This introduces modularity to _CYCLUS_ input files and simulations. 

__XInclude basics__

In order to use `XInclude` simply add the `XInclude` namespace to the root element of the xml file that needs to include another xml file.

> xmlns:xi="http://www.w3.org/2001/XInclude"

Then, reference the xml file that needs to be added to the base xml file using an `href` tag under the `XInclude` namespace.

> `<xi:include href="link_to_xml_file_to_be_added.xml" />`


__Example__

Below is an example of xml inclusion from the final _CYCLUS_ [input file](cyclus/input/UNITED_STATES.xml) (click to open the full file).
```
<simulation xml:base="/home/gyutae/cyclus/predicting-the-past/cyclus/input/"
  xmlns:xi="http://www.w3.org/2001/XInclude">
  <control>
    <duration>1020</duration>
    <startmonth>1</startmonth>
    <startyear>1965</startyear>
  </control>
  <archetypes>
    <spec><lib>cycamore</lib> <name>Enrichment</name> </spec>
    <spec><lib>cycamore</lib> <name>Reactor</name>    </spec>
    <spec><lib>cycamore</lib> <name>Sink</name>       </spec>
    <spec><lib>cycamore</lib> <name>Source</name>     </spec>
    <spec><lib>cycamore</lib> <name>Storage</name>    </spec>
    <spec><lib>cycamore</lib> <name>DeployInst</name> </spec>
    <spec><lib>cycamore</lib> <name>ManagerInst</name></spec>
    <spec><lib>agents</lib>   <name>NullRegion</name> </spec>
  </archetypes>
  <xi:include href="UNITED_STATES/buildtimes/inclusions.xml#xpointer(/inclusions/child::*)"/>
.
.
.
```
As shown above, `XInclude` namespace was added to the root element: `simulation`. The xml file to be added (inclusions.xml) was declared under the namespace using `xi:include` with the relative link `UNITED_STATES/buildtimes/inclusions.xml#xpointer(/inclusions/child::*)`.

__Rationale for setting xml base__

One issue that arose with the use of relative path to reference an external entity in XInclude, was that the xml parser did not know where to find the referenced documents. Thus, parsing the xml from different directory yielded different results and lead to _unable-to-find-external-entity_ errors. Setting an xml base allowed the parser to correctly find the referenced files.

__Rendering final input file__

Setting an xml base was done by finding the absolute path of the _CYCLUS_ input file, and using `jinja2` to render the absolute path. Furthermore, in order to specify all the nations that were included for this simulation, `jinja2` was also used to render the different nations as institutions.

In [ ]:
def render_cyclus(cyclus_template, region, in_dict, out_path):
    """ Renders final cyclus output file with xml base, and institutions
    for each country

    Parameters
    ----------
    cyclus_template: str
        path to cyclus_tempalte
    region: str
        region chosen for cyclus simulation
    in_dict: dictionary
        in_dict should be buildtime_dict from get_buildtime function
    out_path: str
        output path for cyclus input file
    output_name:

    Returns
    -------
    null
        writes cyclus input file in out_path
    """
    if out_path[-1] != '/':
        out_path += '/'
    cyclus_template = idata.load_template(cyclus_template)
    country_list = {value[0].replace(' ', '_') for value in in_dict.values()}
    rendered = cyclus_template.render(countries=country_list,
                                      base_dir=os.path.abspath(out_path) + '/')
    with open(out_path + region + '.xml', 'w') as output:
        output.write(rendered)


cyclus_tmpl = ('templates/' + region + '/' + region + '_template.xml')
render_cyclus(cyclus_tmpl, region, buildtime, 'cyclus/input/')

[UNITED_STATES.xml](cyclus/input/UNITED_STATES.xml) shows the final _CYCLUS_ input file with all the xml inclusions and xml base.

__Running _CYCLUS_ Simulation__

Using the final input file above, a _CYCLUS_ simulation was run using the following command on bash:

In [ ]:
!rm cyclus/UNITED_STATES.sqlite
!cyclus -i ./cyclus/input/UNITED_STATES.xml -o ./cyclus/UNITED_STATES.sqlite --warn-limit 0

The first command removed the previous output file if it exists. This was to prevent the final output file from having the results of the previous simulation, which was unwanted for this investigation. The output file was named [UNITED_STATES.sqlite](cyclus/UNITED_STATES.sqlite). The results of the simulation were stored as an SQLite file. A set of functions were written in python to make sqlite queries. With python, the results of the queries can be processed to perform a meaningful analysis.

## Analysis and Results

The results for the analysis performed above were shown below. Unfortunately, the only data that was available for comparison was the power generated over time, which was published by the Nuclear Energy Institute (NEI) [3], [4]. The data published by the NEI contained power generated by US nuclear power plants from 1971 to 2016. Thus, a plot was generated to show the power generated in that time period in Figure 10. The plot of power generated produced from the data published by the NEI was shown in Figure 11.

__Connect to SQLite file__

To make sqlite queries in _python_, a cursor, which acted like a pointer to the sqlite file was required. The use of `row_factory` class allowed columns in the sqlite table to be indexed by both integer values and by their case-insensitive names. The use of row_factory allowed easy debugging, and prevented confusion that may arise from the change of sqlite queries.

In [ ]:
def get_cursor(file_name):
    """ Connects and returns a cursor to an sqlite output file

    Parameters
    ----------
    file_name: str
        name of the sqlite file

    Returns
    -------
    sqlite cursor
    """
    con = lite.connect(file_name)
    con.row_factory = lite.Row
    return con.cursor()


cursor = get_cursor('cyclus/UNITED_STATES.sqlite')

__Get simulation start time, duration, and timestep__

Since all analysis results were plotted for a visual representation, simulation start time, and timestep were obtained by making an sqlite query using the `get_timesteps` function.

In [ ]:
def get_timesteps(cur):
    """ Returns simulation start year, month, duration and
    timesteps (in numpy linspace).

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    -------
    init_year: int
        start year of simulation
    init_month: int
        start month of simulation
    duration: int
        duration of simulation
    timestep: list
        linspace up to duration
    """
    info = cur.execute('SELECT initialyear, initialmonth, '
                       'duration FROM info').fetchone()
    init_year = info['initialyear']
    init_month = info['initialmonth']
    duration = info['duration']
    timestep = np.linspace(0, duration - 1, num=duration)

    return init_year, init_month, duration, timestep


ini_yr, ini_month, dur, timestep = get_timesteps(cursor)
print('Year: ', ini_yr, '\nMonth: ', ini_month, '\nDuration: ',
      dur, '\nTimestep: ', timestep)

__Nat_u consumption vs Time__

`nat_u_timeseries` function returned a timeseries list of natural uranium sent into the enrichment facility. Unfortunately, this does not represent the amount of natural uranium used in real life as _CYCAMORE::enrichment_ facility did not enrich fuel on a need-basis. Due to current limits in the software, the enrichment facility would always enrich the same amount of natural uranium at each timestep. If _CYCAMORE::enrichment_ were updated to enrich fuel on a need-basis, this analysis would be more accurate. The `nat_u_timeseries` function made an sqlite query on the `timeseriesenrichmentfeed` table from the simulation output file and passed the results to `get_timeseries_cum` function. The `get_timeseries_cum` function calculated the chronological cumulative sum of the natural uranium transferred. The resulting timeseries list was used to plot the cumulative amount of natural uranium used in enrichment over time.

In [ ]:
def nat_u_timeseries(cur):
    """ Finds natural uranium supply from source
        Since currently the source supplies all its capacity,
        the timeseriesenrichmentfeed is used.

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    -------
    get_timeseries: function
        calls a function that returns timeseries list of natural U
        demand from enrichment [MTHM]
    """
    init_year, init_month, duration, timestep = get_timesteps(cur)
    # Get Nat U feed to enrichment from timeseriesenrichmentfeed
    feed = cur.execute('SELECT time, sum(value) '
                       'FROM timeseriesenrichmentfeed '
                       'GROUP BY time').fetchall()

    return get_timeseries_cum(feed, duration, True)


def get_timeseries_cum(in_list, duration, kg_to_tons):
    """ returns a timeseries list from in_list data.

    Parameters
    ----------
    in_list: list
        list of data to be created into timeseries
        list[0] = time
        list[1] = value, quantity
    multiplyby: int
        integer to multiply the value in the list by for
        unit conversion from kilograms
    kg_to_tons: bool
        if True, list returned has units of tons
        if False, list returned as units of kilograms

    Returns
    -------
    timeseries of commodities in kg or tons
    """
    value = 0
    value_timeseries = []
    array = np.array(in_list)
    if len(in_list) > 0:
        for i in range(0, duration):
            value += sum(array[array[:, 0] == i][:, 1])
            if kg_to_tons:
                value_timeseries.append(value * 0.001)
            else:
                value_timeseries.append(value)
    return value_timeseries


consumption = {'Nat_u_consumption': nat_u_timeseries(cursor)}

The resulting timeseries was passed onto `stacked_bar_chart` function for plotting.

In [ ]:
an.stacked_bar_chart(consumption, timestep,
                     'Time [Yr]', 'Nat_u Consumed [MTHM]',
                     'Nat_u consumed vs time',
                     'analysis/results/UNITED_STATES/Nat_u consumption',
                     ini_yr)

Image(filename='analysis/results/UNITEDS/Nat_u consumption.png')

As shown in the figure above, the mass of natural uranium consumed over the period of the simulation linearly increased. This indicated that the enrichment facility enriched fuel at a constant rate throughout the simulation, regardless of the demand for fresh fuel. This will hopefully be improved in the future so that the enrichment facilities also take the demand for fresh fuel into account.

__Amount of fuel into reactors vs. time__

A more accurate analysis for fuel consumption would be the amount of fuel sent to reactors over time. While the enrichment facility constantly produced fresh fuel, the total amount of fuel sent to reactors changed based on the number and size of the reactors in operation at each timestep. This was performed with `fuel_into_reactors` function. The function made an sqlite query to the `resources`, `transactions`, and `agententry` table to obtain the amount of each fuel sent to reactors throughout the simulation. The resulting data was sent to `get_timeseries_cum` function to obtain the timeseries list of different fuels sent to reactors throughout the simulation.

In [ ]:
def fuel_into_reactors(cur):
    """ Finds timeseries of mass of fuel received by reactors

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    -------
    timeseries list of fuel into reactors [tons]
    """
    init_year, init_month, duration, timestep = get_timesteps(cur)
    fuel = cur.execute('SELECT time, sum(quantity) FROM transactions '
                       'INNER JOIN resources ON '
                       'resources.resourceid = transactions.resourceid '
                       'INNER JOIN agententry ON '
                       'transactions.receiverid = agententry.agentid '
                       'WHERE spec LIKE "%Reactor%" '
                       'GROUP BY time').fetchall()

    return get_timeseries_cum(fuel, duration, True)


to_reactor = {'Fuel_to_reactor': fuel_into_reactors(cursor)}
an.stacked_bar_chart(to_reactor, timestep,
                     'Time [Yr]', 'Fuel into Reactors [MTHM]',
                     'Fuel to Reactors over Time',
                     'analysis/results/US/Fuel to Reactors over Time',
                     ini_yr)

Image(filename='analysis/results/US/Fuel to Reactors over Time.png')

As shown in the figure above, the amount of fuel into reactors over time is not linear and changes with fuel demand by the reactors. This figure reflects the state of the reactor (whether it is operating or refueling), and the number of reactors. From the plot, it can be seen that the number of reactors greatly increased around 1990s.

__Fuel utilization vs. time__

Fuel utilization factor is the ratio of fuel spent to the amount of natural uranium consumed. While, fuel utilization factor is a value of interest, it is unfortunately, inaccurate at this moment because the enrichment facilities in _CYCLUS_ does not perform demand-driven fuel enrichment. Since the amount of natural uranium consumed is not accurate due to limitations with software, fuel utilization factor, which is a derived variable, will still be inaccurate. However, the functions used to calculate the fuel utilization factor over time will still be displayed for future usage. This is performed with `u_util_calc` function, which runs `nat_u_timeseries` function and `fuel_into_reactors` function to obtain the timeseries lists of natural uranium consumed and the amount of fuel consumed. Then, and element-wise division of the two lists were performed to obtain the fuel utilization factor.

In [ ]:
def u_util_calc(cur):
    """ Returns fuel utilization factor of fuel cycle

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    -------
    u_util_timeseries: numpy array
        Timeseries of Uranium utilization factor
    Prints simulation average Uranium Utilization
    """
    # timeseries of natural uranium
    u_supply_timeseries = np.array(nat_u_timeseries(cur))

    # timeseries of fuel into reactors
    fuel_timeseries = np.array(fuel_into_reactors(cur))

    # timeseries of Uranium utilization
    u_util_timeseries = np.nan_to_num(fuel_timeseries / u_supply_timeseries)
    print('The Average Fuel Utilization Factor is: ')
    print(sum(u_util_timeseries) / len(u_util_timeseries))

    return u_util_timeseries


fuel_util = {'Fuel Utilization Factor': u_util_calc(cursor)}
an.stacked_bar_chart(fuel_util, timestep,
                     'Time [Yr]', 'Fuel utiliization',
                     'Fuel utilization',
                     'analysis/results/US/Fuel utilization', ini_yr)

Image(filename='analysis/results/US/Fuel utilization.png')

__Capacity vs. time__

Total capacity over time is obtained using `get_power_dict` function and `capacity_calc` function. The `get_power_dict` function performs four sqlite queries to obtain the following: simulation start date, timesteps, institutions declared in _CYCLUS_, and reactors entry and exit timesteps. The results are sent to `capacity_calc` function, which calculates the timeseries value of the total capacity of the reactors in each institution.

In [ ]:
def get_power_dict(cur):
    """ Gets dictionary of power capacity by calling capacity_calc

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    ------
    power_dict: dictionary
        "dictionary with key=government, and
        value=timeseries list of installed capacity"
    """
    init_year, init_month, duration, timestep = get_timesteps(cur)
    governments = get_inst(cur)

    # get power cap values
    entry = cur.execute('SELECT max(value), timeseriespower.agentid, '
                        'parentid, entertime FROM agententry '
                        'INNER JOIN timeseriespower '
                        'ON agententry.agentid = timeseriespower.agentid '
                        'GROUP BY timeseriespower.agentid').fetchall()

    exit_step = cur.execute('SELECT max(value), timeseriespower.agentid, '
                            'parentid, exittime FROM agentexit '
                            'INNER JOIN timeseriespower '
                            'ON agentexit.agentid = timeseriespower.agentid'
                            ' INNER JOIN agententry '
                            'ON agentexit.agentid = agententry.agentid '
                            'GROUP BY timeseriespower.agentid').fetchall()

    return capacity_calc(governments, timestep, entry, exit_step)


def get_inst(cur):
    """ Returns prototype and agentids of institutions

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    -------
    sqlite query result (list of tuples)
    """
    return cur.execute('SELECT prototype, agentid FROM agententry '
                       'WHERE kind = "Inst"').fetchall()


def capacity_calc(governments, timestep, entry, exit_step):
    """Adds and subtracts capacity over time for plotting

    Parameters
    ----------
    governments: list
        list of governments (countries)
    timestep: np.linspace
        list of timestep from 0 to simulation time
    entry: list
        power_cap, agentid, parentid, entertime
        of all entered reactors
    exit_step: list
        power_cap, agentid, parenitd, exittime
        of all decommissioned reactors

    Returns
    -------
    power_dict: dictionary
        "dictionary with key=government, and
        value=timeseries list capacity"
    """
    power_dict = collections.OrderedDict()
    for gov in governments:
        capacity = []
        cap = 0
        for t in timestep:
            for enter in entry:
                if (enter['entertime'] == t and
                        enter['parentid'] == gov['agentid']):
                    cap += enter['max(value)'] * 0.001
            for dec in exit_step:
                if (dec['exittime'] == t and
                        dec['parentid'] == gov['agentid']):
                    cap -= dec['max(value)'] * 0.001
            capacity.append(cap)
        power_dict[gov['prototype']] = np.asarray(capacity)

    return power_dict


capacity_dict = get_power_dict(cursor)

an.stacked_bar_chart(capacity_dict, timestep,
                     'Years', 'Net_Capacity [GWe]',
                     'Net Capacity vs Time',
                     'analysis/results/US/Capacity vs Time',
                     ini_yr)

Image(filename='analysis/results/US/Capacity vs Time.png')

As shown in the figure above, the net capacity greatly increases between 1965 and 1990. The increase in capacity reflects the increase in the number of reactors in operation. Then, the capacity decreases gradually from around 1995. This trend is also shown in the figure of number of reactors over time below.

__Number of reactors vs time__

A very similar procedure is used to obtain the number of reactors over time. The same four queries made to calculate capacity over time were made to get the number of reactors over time. Then, this is passed to `reactor_deployments` functions, which uses a counter to keep track of the number of reactors operating over time. 

In [ ]:
def get_deployment_dict(cur):
    """ Gets dictionary of reactors deployed over time
    by calling reactor_deployments

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    ------
    num_dict: dictionary
        "dictionary with key=government, and
        value=timeseries list of number of reactors"
    """
    init_year, init_month, duration, timestep = get_timesteps(cur)
    governments = get_inst(cur)

    # get power cap values
    entry = cur.execute('SELECT max(value), timeseriespower.agentid, '
                        'parentid, entertime FROM agententry '
                        'INNER JOIN timeseriespower '
                        'ON agententry.agentid = timeseriespower.agentid '
                        'GROUP BY timeseriespower.agentid').fetchall()

    exit_step = cur.execute('SELECT max(value), timeseriespower.agentid, '
                            'parentid, exittime FROM agentexit '
                            'INNER JOIN timeseriespower '
                            'ON agentexit.agentid = timeseriespower.agentid'
                            ' INNER JOIN agententry '
                            'ON agentexit.agentid = agententry.agentid '
                            'GROUP BY timeseriespower.agentid').fetchall()

    return reactor_deployments(governments, timestep, entry, exit_step)


def reactor_deployments(governments, timestep, entry, exit_step):
    """Adds and subtracts number of reactors deployed over time
    for plotting

    Parameters
    ----------
    governments: list
        list of governments (countries)
    timestep: np.linspace
        list of timestep from 0 to simulation time
    entry: list
        power_cap, agentid, parentid, entertime
        of all entered reactors

    exit_step: list
        power_cap, agentid, parenitd, exittime
        of all decommissioned reactors

    Returns
    -------
    deployment: dictionary
        "dictionary with key=government, and
        value=timeseries number of reactors"
    """
    deployment = collections.OrderedDict()
    for gov in governments:
        num_reactors = []
        count = 0
        for t in timestep:
            for enter in entry:
                if (enter['entertime'] == t and
                        enter['parentid'] == gov['agentid']):
                    count += 1
            for dec in exit_step:
                if (dec['exittime'] == t and
                        dec['parentid'] == gov['agentid']):
                    count -= 1
            num_reactors.append(count)
        deployment[gov['prototype']] = np.asarray(num_reactors)

    return deployment


an.stacked_bar_chart(get_deployment_dict(cursor),
                     timestep, 'Years',
                     'Number of Reactors',
                     'Number of Reactors vs Time',
                     'analysis/results/US/Number of Reactors vs Time',
                     ini_yr)

Image(filename='analysis/results/US/Number of Reactors vs Time.png')

__Power generated vs. time__

The amount of power generated over time is can be calculated from capacity over time with a few assumptions. Power generated can be calculated from capacity with the using the following function.

$$P(t)=C(t)\times CF(t)\times HRPYR$$

where,

$$P(t) = Power\ generated\ over\ a\ year\ [GWh]$$
$$C(t) = Capacity\ over\ time\ [GWe]$$
$$CF(t)= Capacity\ factor$$
$$HRPYR= Number\ of\ hours\ in\ a\ year$$

The following assumptions were used for the calculation above.

1. Refueling time of all reactors is 1 month.
2. Capacity factor is constant throughout the year, and equal for all reactors.
3. Reactor generates its full power even during startup and shutdown.
4. Capacity factor data is obtained from the Energy Information Administration (EIA), which has the average capacity factor for reactors in the United States over a year from 1971 to 2017. The capacity factor is assumed constant from 2016 to the end of the simulation and from 1965 to 1971.

Capacity factor for each year is stored in a separate [spreadsheet file](analysis/published_data/US/capacity_factor_extrapolated.csv). A set of python functions were used to calculate the power generated over time.

In [ ]:
hours_in_year = 24 * 365.25
cf_data = import_csv('analysis/published_data/US/capacity_factor_extrapolated.csv',
                     ',')
print(*cf_data[:20], sep='\n')
print('.', '.', '.', sep='\n')

Assumption 4 can be seen from `cf_data`. The capacity factor is assumed to be constant at 48.2% from 1965 to 1971, and at 92.5% from 2017 to 2049. Power generated at each timestep is calculated by performing an element-wise multiplication of  the capacity factor data and the capacity. However, since capacity factor is stored for each year, Assumption 2 is used. A separate function is used to create a list that stores the capacity factor for each timestep of the simulation.

In [ ]:
def get_cf(in_list):
    """ Creates a list of capacity factor from
    the imported csv file

    Parameters
    ----------
    in_list: list
        list containing data stored in a csv file

    Returns
    -------
    cf: list
        list of capacity factors per month
    """
    cf = []
    for row in in_list[1:]:
        for i in range(0, 12):
            cf.append(float(row[1]) / 100)

    return cf


cf = np.asarray(get_cf(cf_data))
capacity = capacity_dict['US']
generated = {"US": np.multiply(capacity, cf) * hours_in_year}
an.stacked_bar_chart(generated, timestep,
                     'Years', 'Power Generated [GWh]',
                     'Power Generated',
                     'analysis/results/US/Power Generated',
                     ini_yr)

Image(filename='analysis/results/US/Power Generated.png')

__Power generated between 1971 and 2016__

The United States Nuclear Energy Institute (NEI) has published the amount of power generated by nuclear power plants in the United States between 1971 and 2016 (US Nuclear Generating Statistics, NEI). This information is used to compare the simulation results with the actual data. Thus, the data obtained from the previous analysis is used to obtain the power generated between 1971 and 2016 by calculating the starting index and the ending index. The actual data is plotted for comparison.

In [ ]:
start_year = 1971
end_year = 2017
start_index = (start_year - 1965) * 12
end_index = (end_year - 1965) * 12
dura = end_year - start_year

timestep_range = timestep[start_index: end_index]
cf_range = cf[start_index: end_index]
generated_range = {"US": generated["US"][start_index: end_index]}

an.stacked_bar_chart(generated_range, timestep_range,
                     "Years", "Power Generated [GWh]",
                     "Power Generated",
                     "analysis/results/US/Power Generated 1971~2016",
                     1965)

Image(filename='analysis/results/US/Power Generated 1971~2016.png')

In [ ]:
Image(filename='analysis/published_data/US/Power Generated NEI.png')

As shown in the two plots above,the plot of power generated between 1971 and 2016 from the _CYCLUS_ simulation matches very closely to that published by the NEI. There are a few differences that can be noticed between the two plots. First, the power generated according to _CYCLUS_ is slightly higher than the power generated according to the NEI. This can be explained from the difference in refueling times between simulation and real-world scenarios. Due to software limitations of _CYCLUS_, refueling time for reactors can only be entered in integers of month, and stays constant throughout the lifetime of the plant whereas actual refueling time can vary in floating points of month and changes depending on various factors.

_CYCLUS_ simulation used to obtain the power generated over time assumes a refueling period of 1 month. According to the NEI, the average  refueling period for reactors in the United States varies quite significantly. The average refueling period in 1990 was 104 days, a period of over 3 months, and generally decreases to an average refueling period of 35 days, a period of just over 1 month, in 2017. As published by the NEI, the average refueling period of nuclear reactors in the United States has been greater than 1 month between 1990 and 2017. This means that the power generated by the reactors deployed in the _CYCLUS_ simulation would have a greater amount of power generated over a period of time.

## Conclusion

To conclude, _CYCLUS_ has been able to perform simulations at a fairly accurate level as shown in the comparison of the power generation plots. Still, some improvements in _CYCLUS_ would be benefitial: the implementation of an enrichment facility that processes fuel on a need basis. There are other improvements that can be potentially made for this simulation to improve its accuracy.

+  Specify actual burnup for reactors
    + Currently, the reactors deployed in Cyclus are all deployed with a burnup of 51 GWd/MTHM. 
    
    
+  Specify reactor n_assembly_core and batch
    + Reactors were deployed in cyclus without the actual number of assemblies per core and batch. The typical values were used for each reactor type (Source needs to be added for the typical values). While the effects of changing these values may not be significant in the outcome of the simulation, specifying such information may benefit the accuracy of the simulation results.
    
    
+  Apply a similar analysis to different regions such as the EU or in the world


## References

[1] N. Todreas and M. Kazimi, Nuclear systems. Boca Raton, FL: CRC Press, 2012.


[2] D. Cacuci, Handbook of nuclear engineering. New York: Springer, 2010.


[3] US Nuclear Refueling Outage Days. (n.d.). Retrieved from https://www.nei.org/Knowledge-Center/Nuclear-Statistics/US-Nuclear-Power-Plants/US-Nuclear-Refueling-Outage-Days


[4] US Nuclear Generating Statistics. (n.d.). Retrieved from https://www.nei.org/Knowledge-Center/Nuclear-Statistics/US-Nuclear-Power-Plants/US-Nuclear-Generating-Statistics